# alphadia-validate
A notebook for exploring DIA hits for the sake of validation.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os

from pathlib import Path
from src.slicer import SpectrumSlicer

from src.isoform_attention import isoform_attention_plot
from src.io import prepare_data
from src.mirror_plotting import mirror_w_xic_w_corrs

from IPython.display import Image

## 0. Set up data

In [ ]:
# this is the folder where the data should be read from / downloaded to
custom_folder = "~//alphadia-validate"

# either download example data ..
download_data = True

# .. or use your own data
# download_data = False
raw_file_name = "20231017_OA2_TiHe_ADIAMA_HeLa_200ng_Evo011_21min_F-40_07.raw" # raw file
precursors_file_name = "precursors.csv" # output of an AlphaDIA search (cf. search.ipynb)
speclib_file_name = "speclib.hdf" # output of an AlphaDIA search (cf. search.ipynb)

In [ ]:
main_folder = Path(os.environ.get("BASE_FOLDER", custom_folder)).expanduser()

if download_data:
    precursor_df, spectral_library, spectral_library_flat, dia_data = prepare_data(
        main_folder, download_data=True
    )
else:
    precursor_df, spectral_library, spectral_library_flat, dia_data = prepare_data(
    main_folder,
    raw_file_name = raw_file_name,
    precursors_file_name = speclib_file_name,
    speclib_file_name = precursors_file_name
)


## 1. Select a precursor

In [ ]:
# user input: select a hash
selected_hash = precursor_df["mod_seq_charge_hash"].iloc[3000]

# alternatively: use a random one from a certain percentile
# from src.small_helpers import get_random_precursor_hash
# selected_hash = get_random_precursor_hash(precursor_df, 0)

## 2. Preprocess

In [ ]:
library_wrapper = SpectrumSlicer(spectral_library_flat, precursor_df, dia_data)

mz_library, intensity_library, spectrum_slice, fragment_library = (
    library_wrapper.get_by_hash(selected_hash)
)

## 3. Plots

### Mirror plots


In [ ]:
mirror_w_xic_w_corrs(
    spectrum_slice,
    mz_library,
    intensity_library,
    fragment_library,
    precursor_df,
    selected_hash,
    width=500,
    height=300,
)

In [ ]:
# this is just for display on GitHub as it does not show altair plots
Image(filename="../misc/mirror.png")

In [ ]:
# plot_mirror_2(spectrum_slice, mz_library, intensity_library, )

### Isoform attention plot

In [ ]:
# in case you want to find some precursors with modifications:
# display(precursor_df[~pd.isna(precursor_df["mods"])][["mod_seq_charge_hash", "mods"]].head(10))

isoform_attention_plot(spectral_library_flat, precursor_df, dia_data, selected_hash)

### Shared fragments
In order to play with that plot, use the `shared_fragment_histogram.ipynb` notebook.

In [ ]:
Image(filename="../misc/shared_fragment_histogram.png")